# Layerwise Sensitivity

See how much different layers are sensitive to (1) pairs with dobj/iobj change, and (2) random sentence pairs.

In [1]:
import sys
sys.path.append('../')

from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import pandas as pd
import os, sys, time, re
import matplotlib.pyplot as plt
import random
import pickle
from scipy.spatial.distance import cosine
import seaborn as sns

import src.sent_encoder
import src.sentpair_generator

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Load sentence pairs

In [2]:
with open('../data/sents.pkl', 'rb') as f:
  data = pickle.load(f)
  data = list(data)

In [3]:
# https://wortschatz.uni-leipzig.de/en/download/english
# Wikipedia, 2016, 10K sentences
sentgen = src.sentpair_generator.SentPairGenerator()
wiki_sents = sentgen.get_wikipedia()

In [4]:
len(data)

321

In [5]:
enc = src.sent_encoder.SentEncoder()

## Generate boxplots

In [ ]:
model_name = 'roberta-base'
sns.boxplot(x='layer', y='dist', data=distances)
plt.title(f"Sentence embedding distance for dobj/iobj pairs.\nmodel: {model_name}")
plt.ylim(0)
plt.show()

In [6]:
wiki_sents = sentgen.get_wikipedia()
wiki_distances = enc.get_layer_distance_df(wiki_sents)

Processing layer: 0
499875840 499875840
Processing layer: 1
499875840 2004178944
Processing layer: 2
499875840 2204964864
Processing layer: 3
499875840 2425521152
Processing layer: 4
499875840 2732648960
Processing layer: 5
499875840 3060992512
Processing layer: 6
499875840 3389679104
Processing layer: 7
499875840 3725683200
Processing layer: 8
499875840 4053210112
Processing layer: 9
499875840 4382019584
Processing layer: 10
499875840 4712297472
Processing layer: 11
499875840 5043292672
Processing layer: 12
499875840 5372537344


In [7]:
sns.boxplot(x='layer', y='dist', data=wiki_distances)
plt.title(f"Sentence embedding distance for wiki pairs.\nmodel: {model_name}")
plt.ylim(0)
plt.show()

ValueError: Could not interpret input 'layer'

## Combined plot

In [ ]:
distances['dataset'] = 'dobj/iobj'
wiki_distances['dataset'] = 'random wiki'
combined_df = pd.concat([distances, wiki_distances])

In [ ]:
sns.barplot(x='layer', y='dist', hue='dataset', data=combined_df)
plt.title(f"model: {model_name}")
plt.ylim(0)
plt.show()